# Creating the dataset

In [ ]:
!rsync --recursive --verbose --include='*/' --include='posts000000000011*' --include='tags*' --exclude='*' rsync://176.9.41.242:873/danbooru2021/metadata/ ./danbooru2021/

### Making the list of tags

In [1]:
import pandas as pd

In [ ]:
orig_tags = pd.read_json("danbooru2021/tags000000000000.json", lines=True)
tags = orig_tags.copy()
tags = tags[(tags['category'] == 0) & (tags['post_count'] >= 600)].sort_values(by='post_count', ascending=False)
tags[['name','post_count']]

In [ ]:
# Saving it to a txt file separated by \n
tags.name.to_csv('data/tags.txt', index=False, header=False)

### Filtering posts

In [ ]:
VIEW = ['tag_string_general', 'tag_count_general', 'score', 'rating']
orig_posts = pd.read_json("danbooru2021/posts000000000011.json", lines=True)
posts = orig_posts.copy()
posts = posts[(posts['tag_count_general'] > 12)].sample(50000, random_state=1337)
posts[VIEW]

In [ ]:
tag_list = open("data/tags.txt").read().split('\n')

In [ ]:
# Function to filter tags
def filter_tags(tag_string):
    tags = tag_string.split()
    filtered_tags = [tag for tag in tags if tag in tag_list]
    return ' '.join(filtered_tags)

# Apply the filtering function to create the new column
posts['tag_string_filtered'] = posts['tag_string_general'].apply(filter_tags)
posts['tag_string_filtered']

In [ ]:
posts.to_json("data/posts.json", orient="records", lines=True)

### Generating the captions

In [ ]:
%pip install langchain openai nest_asyncio tqdm -Uqq

In [2]:
import pandas as pd
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain
from tqdm import tqdm
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
posts = pd.read_json("data/posts.json", lines=True)

In [4]:
template = """
Instructions:Write captions for an illustration based on the given keywords.
tags:1girl bangs black_hair black_serafuku blunt_bangs hime_cut long_hair long_sleeves school_uniform serafuku solo
caption:a single girl with long straight black hair with blunt bangs styled into a princess cut wearing a black school uniform with long sleeves
tags:1girl alternate_costume breasts china_dress chinese_clothes cleavage dress greyscale large_breasts lips monochrome pelvic_curtain sketch solo squatting
caption:A monochrome sketch of a single woman wearing an alternate costume of a traditional Chinese dress with a pelvic curtain, squatting on the ground with her large breasts exposed. She has her lips slightly parted in a coy smile.
tags:{tags}
caption: 
""".strip()

prompt = PromptTemplate(template=template, input_variables=["tags"])

In [5]:
llm = OpenAI()
chain = LLMChain(prompt=prompt, llm=llm)

In [6]:
async def process_batch(batch):
    results = []
    # Create a list of tasks for concurrent execution
    tasks = [chain.arun(item) for item in batch]
    # Run the tasks concurrently using asyncio.gather
    processed_results = await asyncio.gather(*tasks)
    # Add the processed results to the results list
    results.extend(processed_results)
    return results

In [8]:
async def process_batches(batches):
    captions = []
    for batch in batches:
        try:
            results = await process_batch(batch[0])
            captions.extend(results)
            await asyncio.sleep(10)
        except:
            # Handle rate limit exceeded error
            await asyncio.sleep(10)
    return captions

In [11]:
asyncio.run(process_batches(test_batches))

ValueError: too many file descriptors in select()

: 

In [9]:
batch_size = 100  # You can adjust this to your desired batch size

# Generate batches
batches = []
for i in range(0, len(posts['tag_string_filtered']), batch_size):
    batches.append(posts['tag_string_filtered'][i:i + batch_size].to_list())

In [10]:
test_batches = []
for i in range(0, 100, 10):
    batch = batches[0][i:i+10]
    test_batches.append(batch)

In [ ]:
captions = []
for x in test_batches:
    results = asyncio.run(process_batch(x[0]))
    captions.extend(results)
    await asyncio.sleep(1)

pd.Series(captions)

ValueError: too many file descriptors in select()

: 

In [11]:
response = completion_with_backoff(model="gpt-3.5-turbo", prompt=make_prompt(test))

API Error: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?


In [ ]:
test = posts.iloc()[:100]
data = list(test.prompt.values)

### Testing

In [ ]:
tester = posts.iloc()[:10]

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(main(tester['tag_string_general']))

In [ ]:
tester.to_json("tester.json", lines=True, orient="records")

In [ ]:
from google.colab import files
files.download('tester.json')

In [ ]:
import warnings
warnings.filterwarnings(action='once')

tags_series = posts['tag_string_general']
loop = asyncio.get_event_loop()
results = loop.run_until_complete(process_async(tags_series))
posts['caption_string'] = results
posts['tag_string_filtered'] = posts['tag_string_general'].apply(filter_tags)
posts.to_json("posts.json", lines=True, orient="records")
files.download('posts.json')

In [ ]:
results